In [1]:
from sportsipy.ncaab.teams import Teams
from sportsipy.ncaab.schedule import Schedule
from sportsipy.ncaab.boxscore import Boxscore
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas.io import gbq
import os

### Connecting to Google Big Query

In [2]:
#GBQ Credentials

private_key = service_path + service_file

In [3]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

### Gathering Boxscore indexes for each game and each team.

Output: CSV file (schedule_season.csv)

In [ ]:
#gathering teams schedules and appending to a list
#goal is to append to dataframe 

seasons = list(range(2010,2022,1))

game_array = []
for season in seasons:
    teams = Teams(season)
    schedule_df = None
    for team in teams:

        schedule = team.schedule

        for game in schedule:
            ind_game = game.dataframe
            if schedule is not None:
                schedule_df = pd.concat([schedule_df,ind_game], axis = 0)
            else:
                schedule_df = ind_game
                print(schedule)


    #creating file path and output file
    #turing shedules into CSV files
    outname = f'{season}.csv'

    outdir = './seasons_dfs'
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)    

    schedule_df.to_csv(fullname)
        
    table = str(season)
        
        
    #sending schedule files to Google Big Query    
    schedule_df.to_gbq(destination_table= 'schedules.'+ table,
                        chunksize= 500000,
                        project_id= gbq_proj_id,
                        if_exists='replace')
                
    
                
#                 boxscore = game.boxscore_index
#                 date = game.datetime
#                 location = game.location
#                 opponent_conference = game.opponent_conference
#                 opponent_name = game.opponent_name
#                 win_streak = game.streak
#                 overtimes = game.overtimes
#                 reg_season = game.type
#                 game_array.append([season, date, team.name,boxscore, location, opponent_conference, opponent_name, win_streak
#                                   overtimes, reg_season])

#     schedule = pd.DataFrame(game_array, columns=['Season','TeamName','BoxscoreIndex'])
#     schedule.to_csv(f'/seasons_dfs/{season}.csv')
#     table= str(season)
#     schedule.to_gbq(destination_table= 'schedules.'+ table,
#                  chunksize= 500000,
#                  project_id= gbq_proj_id,
#                  if_exists='replace')

### Creating a list of names for every teams schedue 

This list contains the season, team name and boxscore index. 

In [ ]:
temp_stat_2011 = pd.read_csv("2011.csv")

In [ ]:
season = 2011
outname = f'{season}.csv'

path = os.getcwd()
outdir = f'{path}/seasons_dfs'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

temp_stat_2011.to_csv(fullname)

In [ ]:
list_of_schedules = []
path = os.getcwd()
file_list = os.listdir(f'{path}/seasons_dfs")
for file_name in file_list:
    if 'csv' in file_name:
        list_of_schedules.append(file_name)

In [ ]:
list_of_schedules

### Reading in each games boxscore

Output: to google big query. 

Example boxscore is in example_box_score.csv

In [ ]:
for season in list_of_schedules:
    boxscores = None
    error_df = None
    schedule = pd.read_csv('./seasons_dfs/' + season)
    for idx, row in schedule.iterrows():
        try:
            bs_index = row['boxscore_index']
            ind_box = Boxscore(bs_index).dataframe

            if boxscores is not None:
                boxscores = pd.concat([boxscores,ind_box], axis = 0)
            else:
                boxscores = ind_box

        except:

            if error_df is not None:
                error_df = pd.concat([error_df,row], axis = 0)
            else:
                error_df = row

            continue

    
    
    boxscores['away_free_throw_percentage'] = boxscores['away_free_throw_percentage'] * 100

    cols = boxscores.drop(labels = ['date', 'location', 'losing_abbr', 'losing_name',
                                    'winner', 'winning_abbr' , 'winning_name'], axis = 1)

    for col in cols:
        boxscores[col] = boxscores[col].apply(pd.to_numeric, downcast="float")

    boxscores['date'] = pd.to_datetime(boxscores['date'])

    boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                     chunksize= 500000,
                     project_id= gbq_proj_id,
                     if_exists='replace')
    
    
    table = f"BS_{season}"
    
    boxscores.to_csv(table + ".csv")
    
    try:
        
        errodf.to_csv(table + "error.csv")
    
    except:
        print("no error df")


    try:
        
        boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                         chunksize= 500000,
                         project_id= gbq_proj_id,
                         if_exists='replace')
    
    except:
        print("No GBQ Upload :(")
        continue
        
#     errordf.to_gpq(destination_table= 'errors.boxscores',
#                      chunksize= 500000,
#                      project_id= gbq_proj_id,
#                      if_exists='append')

In [ ]:
boxscores.to_csv("boxscores_csv")

In [ ]:
boxscores['date']

In [ ]:
table = f"BS_2019"

boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                 chunksize= 500000,
                 project_id= gbq_proj_id,
                 if_exists='replace')

In [ ]:
errordf.to_gpq(destination_table= 'boxscores.errors', 
               chunksize= 500000,
               project_id= gbq_proj_id,
               if_exists='append')

### Load an individual season

In [ ]:
schedule_df = pd.read_csv('schedule.2010.csv')
schedule_df.head(1)

In [ ]:
#pd.read_csv("./seasons_dfs/2011.csv")

In [7]:
season = 2016 #change
boxscores = None
error_df = None

individual_season = pd.read_csv("./seasons_dfs/2016.csv") #change

for idx, row in individual_season.iterrows():
    try:
        bs_index = row['boxscore_index']
        ind_box = Boxscore(bs_index).dataframe

        if boxscores is None:
            boxscores = ind_box
        else:
            boxscores = pd.concat([boxscores,ind_box], axis = 0)

    except:

        if error_df is None:
            error_df = row
        else:
            error_df = pd.concat([error_df,row], axis = 0)
        
        continue


boxscores['away_free_throw_percentage'] = boxscores['away_free_throw_percentage'] * 100
boxscores['home_free_throw_percentage'] = boxscores['home_free_throw_percentage'] * 100

cols = boxscores.drop(labels = ['date', 'location', 'losing_abbr', 'losing_name',
                                'winner', 'winning_abbr' , 'winning_name'], axis = 1)

for col in cols:
    boxscores[col] = boxscores[col].apply(pd.to_numeric, downcast="float")

boxscores['date'] = pd.to_datetime(boxscores['date'])       

table = f"BS_{season}"
    
boxscores.to_csv(table + ".csv")

boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                 chunksize= 500000,
                 project_id= gbq_proj_id,
                 if_exists='replace')

# errordf.to_gpq(destination_table= 'errors.boxscores',
#                  chunksize= 500000,
#                  project_id= gbq_proj_id,
#                  if_exists='append')

1it [00:18, 18.91s/it]


In [6]:
table = f"BS_{season}"
    
boxscores.to_csv(table + ".csv")

boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                 chunksize= 500000,
                 project_id= gbq_proj_id,
                 if_exists='replace')

1it [00:16, 16.92s/it]


In [ ]:
newbox = pd.read_csv('BS_2020.csv', index_col = 0)
table = f"BS_2020"

In [ ]:
boxscores['away_free_throw_percentage'] = boxscores['away_free_throw_percentage'] * 100

cols = boxscores.drop(labels = ['date', 'location', 'losing_abbr', 'losing_name',
                                'winner', 'winning_abbr' , 'winning_name'], axis = 1)

for col in cols:
    boxscores[col] = boxscores[col].apply(pd.to_numeric, downcast="float")

boxscores['date'] = pd.to_datetime(boxscores['date'])

boxscores['losing_name'] = boxscores['losing_name'].map(lambda x: x.rstrip('\n\t\t\t'))
boxscores['losing_abbr'] = boxscores['losing_abbr'].map(lambda x: x.rstrip('\n\t\t\t'))
boxscores['winning_name'] = boxscores['winning_name'].map(lambda x: x.rstrip('\n\t\t\t'))
boxscores['winning_abbr'] = boxscores['winning_abbr'].map(lambda x: x.rstrip('\n\t\t\t'))

boxscores.to_gbq(destination_table= 'boxscores.'+ table,
                 chunksize= 500000,
                 project_id= gbq_proj_id,
                 if_exists='replace')

In [ ]:
#newbox

In [ ]:
newbox['away_free_throw_percentage'] = newbox['away_free_throw_percentage'] * 100

cols = newbox.drop(labels = ['date', 'location', 'losing_abbr', 'losing_name', 
                             'winner', 'winning_abbr' , 'winning_name'], axis = 1)

for col in cols:
    newbox[col] = newbox[col].apply(pd.to_numeric, downcast="float")

newbox['date'] = pd.to_datetime(newbox['date'])

newbox.to_gbq(destination_table= 'boxscores.'+ table,
              chunksize= 500000,
              project_id= gbq_proj_id,
              if_exists='replace')

In [ ]:
convert_dict = {'away_assist_percentage': float,
                'away_assists': float,
                'away_block_percentage': float,
                'away_blocks': float,
                'away_defensive_rating': float,
                'away_defensive_rebound_percentage': float
                'away_defensive_rebounds': float,
                'away_effective_field_goal_percentage': float,
                'away_field_goal_attempts': float,
                'away_field_goal_percentage': float,
                'away_field_goals': float,
                'away_free_throw_attempt_rate': float,
                'away_free_throw_attempts': float,
                'away_free_throw_percentage': float,
                'away_free_throws': float,
                'away_losses': float,
                'away_minutes_played': float,
                'away_offensive_rating'
               }

In [ ]:
box_df = None
#season is saving as global enviroment.



for season in seasons:
    season_df = schedule_df.loc[schedule_df.Season==season]
    for idx, row in season_df.iterrows():
        try:
            box_link = row['BoxscoreIndex']
            _df = Boxscore(box_link).dataframe

            if box_df is not None:
                box_df = pd.concat([box_df,_df],axis=0)
            else:
                box_df = _df
        except:
            continue
box_df.to_csv(f'{season}_boxscores.csv',index=None)

In [ ]:
for season in seasons:
    print(seasons)

In [ ]:
box_link = '2021-12-04-20-gonzaga'
games = Boxscore(box_link).dataframe
games.to_csv('example_box_score.csv')


In [ ]:
season = 2011 # will run once for each season
box_df = None
schedule_df = pd.read_csv('./output/schedule.csv')
season_df = schedule_df.loc[schedule_df.Season==season]
for index, row in tqdm(season_df.iterrows()):
    box_link = row['BoxscoreIndex']
    _df = Boxscore(box_link).dataframe
    
    if box_df is not None:
        box_df = pd.concat([box_df,_df],axis=0)
    else:
        box_df = _df
box_df.to_csv('./output/{}_boxscores.csv'.format(season),index=None)